<a href="https://colab.research.google.com/github/lblogan14/deep_learning_for_computer_vision/blob/master/ch2_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My' 'Drive/Colab' 'Notebooks/Deep_Learning_for_Computer_Vision/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Deep_Learning_for_Computer_Vision


**Image classification** is the task of classifying a whole image as a single label.

#The MNIST datasets
has handwritten digits from 0-9 with 60,000 images for training and 10,000 images for testing. The images are normalized to the size of 28 image pixels by 28
image pixels, converted to grey size, and centered to a fixed size. This is a small
dataset on which an algorithm can be quickly tested.

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#Perceptron
A single-layer neural network.

In [0]:
input_size = 784
no_classes = 10
batch_size = 100
total_batches = 200

x_input = tf.placeholder(tf.float32, shape=[None, input_size])
y_input = tf.placeholder(tf.float32, shape=[None, no_classes])

The **None** in the `shape` argument indicates it can be of any size as we have not yet defined the batch size.

The second argument is the size of the tensor for `x_input` and the number of classes for `y_input`.

##Define the variables for a fully connected layer

In [0]:
weights = tf.Variable(tf.random_normal([input_size, no_classes]))
bias = tf.Variable(tf.random_normal([no_classes]))

The initialization of the variables can be zeroes but a random normal distribution
gives a steady training.

In [0]:
logits = tf.matmul(x_input, weights) + bias

The inputs are weighted and added with the bias to produce `logits`.

The `logits` produced by the perceptron has to be compared against one-hot labels `y_input`. For multi-label classification, it is better to use softmax coupled with cross-entropy for comparing `logits` and one-hot labels. That is why we use `tf.nn.softmax_cross_entropy_with_logits` here.

In [0]:
softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_input, logits=logits)

The loss can be computed by averaging the cross-entropies. Then the cross-entropy is fed through gradient descent optimization done by `tf.train.GradientDescentOptimizer`. The optimizer takes the loss and minimizes it with a learning rate of `0.5`.

In [0]:
loss_operation = tf.reduce_mean(softmax_cross_entropy)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(loss_operation)

The softmax and cross-entropies are computed together from the `tf.nn` package,
which has several other useful methods. The `tf.train` has several optimizers, and here, we are using the vanilla gradient descent.

The Adam optimizer is particularly useful for computer vision applications. It generally converges faster and we need not define a learning rate to start with.

##Train the model with data
After defining the model and training operation, the next step is to start training the model with datasets. During training, the gradients are calculated and the weights are updated. The variables have not yet been initialized. 

Start the session and initialize the variables using a global variable initializer:

In [0]:
session = tf.Session()
session.run(tf.global_variables_initializer())

The preceding two lines are mostly required to run in order to generate the computational graph in TensorFlow.

Now the
graph is ready to be fed with data and start training. Over a loop, read the data in
batches and train the model. The optimizer has to be called in order for the
graph to update the weights.

In [0]:
for batch_no in range(total_batches):
  mnist_batch = mnist_data.train.next_batch(batch_size)
  train_images, train_labels = mnist_batch[0], mnist_batch[1]
  _, loss_value = session.run([optimizer, loss_operation], 
                              feed_dict={x_input: train_images,
                                         y_input: train_labels})
  print(loss_value)

15.425655
12.528323
10.212998
9.878272
8.96583
9.350922
8.801382
7.9197383
6.283884
7.368129
7.187389
5.4572115
5.6831303
4.525123
4.8468604
4.07745
5.152235
4.097923
4.1719036
4.221296
3.9882026
3.583871
2.7037687
2.7459028
3.4026573
3.5995405
3.7862546
2.7969053
3.0131173
2.8292258
3.6021082
2.5368512
2.8336418
2.6284442
2.4189205
3.0058658
2.3350453
1.8733461
1.8531623
2.382483
2.377443
2.131122
1.7753848
3.292028
2.3170593
2.3126333
2.231833
2.161497
2.5463414
1.322619
2.1830735
2.2246692
2.0370018
1.9545343
2.0228379
1.1891007
1.7369035
2.158346
1.9418837
2.1336083
1.633956
1.110815
1.5804164
1.4515866
1.4303087
1.9055177
1.4277269
1.5750102
1.38172
1.5686188
1.2858803
1.2129692
1.6078556
1.6003119
1.512208
2.433582
1.8361506
1.1280355
1.6376735
1.696698
1.4147207
1.1470606
1.907039
1.9964045
2.0070803
1.236176
1.1887876
0.9960818
1.0750276
1.4116774
0.9897826
1.1516329
1.654535
1.4748168
1.3473212
1.6851028
1.2305746
1.8957536
1.5888234
1.6476947
1.3139136
1.6029781
1.628187
1.57

The first argument of the `run` method of the `session` function can have an array for which the outputs of
the values are requested. We pass loss because printing loss tells us whether the
model is getting trained or not. The loss is expected to decrease as we are
minimizing the loss.

After training, we can evaluate how well the model worked by computing the accuracy.

In [0]:
predictions = tf.argmax(logits, 1)
correct_predictions = tf.equal(predictions, tf.argmax(y_input, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))


The prediction should be the index of the maximum activation. It should be
compared with the ground truth on MNIST labels for correct predictions. The
accuracy is calculated using the average of correct predictions. The accuracy of
the data can be evaluated by running the session with test data as the feed
dictionary.

In [0]:
test_images, test_labels = mnist_data.test.images, mnist_data.test.labels
accuracy_value = session.run(accuracy_operation, 
                             feed_dict={x_input: test_images,
                                        y_input: test_labels})
print('Accuracy: ', accuracy_value)
session.close()

Accuracy:  0.8097


#Multiilayer Convolutional Network
Create a multilayer convolutional network in
TensorFlow and watch how a deeper network improves classification accuracy.

In this section, we will use TensorBoard for visualizing the training process.
In order to visualize the statistics of the variables, the values of variable statistics
have to be added to `tf.summary`.

In [0]:
tf.reset_default_graph()

In [0]:
x_input = tf.placeholder(tf.float32, shape=[None, input_size])
y_input = tf.placeholder(tf.float32, shape=[None, no_classes])


In [0]:
def add_variable_summary(tf_variable, summary_name):
  with tf.name_scope(summary_name + '_summary'):
    mean = tf.reduce_mean(tf_variable)
    tf.summary.scalar('Mean', mean)
    with tf.name_scope('standard_deviation'):
      standard_deviation = tf.sqrt(tf.reduce_mean(tf.square(tf_variable - mean)))
    tf.summary.scalar('StandardDeviation', standard_deviation)
    tf.summary.scalar('Maximum', tf.reduce_max(tf_variable))
    tf.summary.scalar('Minimum', tf.reduce_min(tf_variable))
    tf.summary.histogram('Histogram', tf_variable)

The variable `summary` function writes the summaries of a variable. There are five
statistics added to the summaries: mean, standard deviation, maximum,
minimum and histogram. Summaries can be either a `scalar` or a `histogram`.

Unlike fully-connected network, we wil resize the `MNIST` data intor a square matrix and use it like a 2D image.

In [0]:
x_input_reshape = tf.reshape(x_input, [-1, 28, 28, 1], name='input_reshape')

The dimension -1 denotes that the batch size can be any number. The `name` argument will be reflected in the TensorBoard graph for ease of understanding.

Define the convolution layer,

In [0]:
def convolution_layer(input_layer, filters, kernel_size=[3,3], activation=tf.nn.relu):
  layer = tf.layers.conv2d(inputs=input_layer,
                           filters=filters,
                           kernel_size=kernel_size,
                           activation=activation)
  
  add_variable_summary(layer, 'convolution')
  return layer

The summaries are
added to the layer within the function and the layer is returned.

Next, define pooling layer,

In [0]:
def pooling_layer(input_layer, pool_size=[2,2], strides=2):
  layer = tf.layers.max_pooling2d(inputs=input_layer,
                                 pool_size=pool_size,
                                 strides=strides)
  
  add_variable_summary(layer, 'pooling')
  return layer

Move on and define the fully connected layer,

In [0]:
def dense_layer(input_layer, units, activation=tf.nn.relu):
  layer = tf.layers.dense(inputs=input_layer,
                          units=units,
                          activation=activation)
  add_variable_summary(layer, 'dense')
  return layer

The `pooling_layer` takes the feature map from the convolution layer and reduces it to half its size by skipping, using the pool size
and strides. All these layers are connected as a graph and are just defined. None
of the values is initialized. Another convolution layer can be added to transform
the sampled features from the first convolution layer to better features. After
pooling, we may reshape the activations to a linear fashion in order to be fed
through dense layers:

In [0]:
convolution_layer_1 = convolution_layer(x_input_reshape, 64)
pooling_layer_1 = pooling_layer(convolution_layer_1)
convolution_layer_2 = convolution_layer(pooling_layer_1, 128)
pooling_layer_2 = pooling_layer(convolution_layer_2)
flattened_pool = tf.reshape(pooling_layer_2, [-1, 5*5*128], name='flattened_pool')
dense_layer_bottleneck = dense_layer(flattened_pool, 1024)

The only difference between the convolution layers is the filter size. It's
important that the dimensions change appropriately from layer to layer.
Choosing the parameters for kernel and stride are arbitrary and these numbers
are chosen by experience. Two convolution layers are defined, and this can be
followed by a fully connected layer. A dense-layer API can take any vector of a
single dimension and map it to any number of hidden units, as in this case is 1024.

Variable summaries are added for this layer as well.

The dense layer is followed by a dropout layer with a rate of dropping out. Keeping this high will stop the network from learning. The training mode can be set to `True` and `False` based on when we use this. We will set this as `True` (default is `False`) for the
training. We will have to change this while the accuracy is calculated.

In [0]:
dropout_bool = tf.placeholder(tf.bool)
dropout_layer = tf.layers.dropout(inputs=dense_layer_bottleneck,
                                  rate=0.4,
                                  training=dropout_bool)

The dropout layer is fed again to a dense layer, which is called logits. Logits is
the final layer with activations leading to the number of classes. The activations
will be spiked for a particular class, which is the target class, and can be obtained
for a maximum of those 10 activations:

In [0]:
logits = dense_layer(dropout_layer, no_classes)

Pass the logits through the softmax layer and calculate the cross-entropy.

Use a scope name to get a better visualization in TensorBoard,

In [0]:
with tf.name_scope('loss'):
  softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_input,
                                                                  logits=logits)
  loss_operation = tf.reduce_mean(softmax_cross_entropy, name='loss')
  tf.summary.scalar('loss', loss_operation)

This `loss` function can be optimized with `tf.train` APIs' methods.

In [0]:
with tf.name_scope('optimizer'):
  optimizer = tf.train.AdamOptimizer().minimize(loss_operation)

Define the accuracy,

In [0]:
with tf.name_scope('accuracy'):
  with tf.name_scope('correct_prediction'):
    predictions = tf.argmax(logits, 1)
    correct_predictions = tf.equal(predictions, tf.argmax(y_input, 1))
  with tf.name_scope('accuracy'):
    accuracy_operation = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    
tf.summary.scalar('accuracy', accuracy_operation)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

A scalar summary for accuracy is also added.

Next, start a session and initialize the variables.

In [0]:
session = tf.Session()
session.run(tf.global_variables_initializer())

The summaries have to be merged, and the files for writing the training and
testing summaries have to be defined:

In [0]:
merged_summary_operation = tf.summary.merge_all()
train_summary_writer = tf.summary.FileWriter('./tmp/train', session.graph)
test_summary_writer = tf.summary.FileWriter('./tmp/test')

Note that the graph is written once with the `summary_writer`.

Next, load the data in batches and start the training phase,

In [0]:
test_images, test_labels = mnist_data.test.images, mnist_data.test.labels

for batch_no in range(total_batches):
  mnist_batch = mnist_data.train.next_batch(batch_size)
  train_images, train_labels = mnist_batch[0], mnist_batch[1]
  _, merged_summary = session.run([optimizer, merged_summary_operation],
                                  feed_dict={x_input:train_images,
                                             y_input:train_labels,
                                             dropout_bool:True})
  train_summary_writer.add_summary(merged_summary, batch_no)
  
  if batch_no % 10 == 0:
    merged_summary, _ = session.run([merged_summary_operation, accuracy_operation],
                                    feed_dict={x_input:test_images,
                                               y_input:test_labels,
                                               dropout_bool:False})
    test_summary_writer.add_summary(merged_summary, batch_no)

Summaries are returned in every iteration for training data and are added to the
writer. For every tenth iteration, the test summaries are added. Note that the
dropout is enabled only during training and not during testing.

#Use TensorBoard to Visualize the Network
Please consult the method shown in my website to set up the Tensorboard in Colab
https://sites.google.com/ncsu.edu/binliu14/computational-intelligence?authuser=0

In [0]:
# install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-11-28 21:09:48--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.86.186.182, 52.2.175.150, 52.201.75.180, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.86.186.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]   5.11M  15.7MB/s    in 0.3s    

2018-11-28 21:09:48 (15.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
# run TensorBoard,
# locating the summary file
# training summary -> ./tmp/train
# testing summary -> ./tmp/test
LOG_DIR = './tmp'
get_ipython().system_raw(
      'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(LOG_DIR))

In [0]:
# run ngrok
# run ngrok to tunnel TensorBoard port 6006 to the outside world
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
# Get URL
# access the colab TensorBoard web page
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://f57e6b17.ngrok.io


#Train MNIST Model in Keras
Build the same model using the `tf.keras` APIs

In [0]:
# reset graph
tf.reset_default_graph()
tf.keras.backend.clear_session()

In [0]:
batch_size = 128
no_classes = 10
epochs = 50
image_height, image_width = 28, 28

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], image_height, image_width, 1)
x_test = x_test.reshape(x_test.shape[0], image_height, image_width, 1)
input_shape = (image_height, image_width, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

y_train = tf.keras.utils.to_categorical(y_train, no_classes)
y_test = tf.keras.utils.to_categorical(y_test, no_classes)

In [0]:
def simple_cnn(input_shape):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Conv2D(filters=64,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   input_shape=input_shape)
           )
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Conv2D(filters=128,
                                   kernel_size=(3,3),
                                   activation='relu')
           )
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
  model.add(tf.keras.layers.Dropout(rate=0.4))
  model.add(tf.keras.layers.Dense(units=no_classes, activation='softmax'))
  
  model.summary()
  model.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
  return model

In [0]:
simple_cnn_model = simple_cnn(input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              3277824   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
__________

In [0]:
simple_cnn_model.fit(x_train, y_train, batch_size, epochs, (x_test, y_test))
train_loss, train_accuracy = simple_cnn_model.evaluate(x_train,
                                                       y_train,
                                                       verbose=0)
print('Train data loss: ', train_loss)
print('Train data accuracy: ', train_accuracy)

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
Train data loss:  0.0009889797066601432
Train data accuracy:  0.9998


In [0]:
test_loss, test_accuracy = simple_cnn_model.evaluate(x_test,
                                                     y_test,
                                                     verbose=0)
print('Test data loss: ', test_loss)
print('Test data accuracy: ', test_accuracy)

Test data loss:  0.048683962830335555
Test data accuracy:  0.9931


#Deep Learning Models

##AlexNet
https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf

##VGG-16
stands for **Visual Geometry Group** from Oxford

##Google Inception-V3
https://arxiv.org/pdf/1409.4842.pdf

##Microsoft ResNet-50
https://arxiv.org/pdf/1512.03385.pdf
 
##SqueezeNet
https://arxiv.org/pdf/1602.07360.pdf

##Spatial Transformer Networks
https://arxiv.org/pdf/1506.02025.pdf

##DenseNet
extension of ResNet

https://arxiv.org/pdf/1608.06993.pdf

#Model for cats vs dogs
Prepare and train a model to predict cats versus dogs

##Prepare dataset

In [0]:
import os
import shutil

In [0]:
work_dir = './data/ch2/'

In [10]:
image_names = sorted(os.listdir(os.path.join(work_dir,'train')))
image_names
# make sure the filenames are correct

['cat.0.jpg',
 'cat.1.jpg',
 'cat.10.jpg',
 'cat.100.jpg',
 'cat.1000.jpg',
 'cat.10000.jpg',
 'cat.10001.jpg',
 'cat.10002.jpg',
 'cat.10003.jpg',
 'cat.10004.jpg',
 'cat.10005.jpg',
 'cat.10006.jpg',
 'cat.10007.jpg',
 'cat.10008.jpg',
 'cat.10009.jpg',
 'cat.1001.jpg',
 'cat.10010.jpg',
 'cat.10011.jpg',
 'cat.10012.jpg',
 'cat.10013.jpg',
 'cat.10014.jpg',
 'cat.10015.jpg',
 'cat.10016.jpg',
 'cat.10017.jpg',
 'cat.10018.jpg',
 'cat.10019.jpg',
 'cat.1002.jpg',
 'cat.10020.jpg',
 'cat.10021.jpg',
 'cat.10022.jpg',
 'cat.10023.jpg',
 'cat.10024.jpg',
 'cat.10025.jpg',
 'cat.10026.jpg',
 'cat.10027.jpg',
 'cat.10028.jpg',
 'cat.10029.jpg',
 'cat.1003.jpg',
 'cat.10030.jpg',
 'cat.10031.jpg',
 'cat.10032.jpg',
 'cat.10033.jpg',
 'cat.10034.jpg',
 'cat.10035.jpg',
 'cat.10036.jpg',
 'cat.10037.jpg',
 'cat.10038.jpg',
 'cat.10039.jpg',
 'cat.1004.jpg',
 'cat.10040.jpg',
 'cat.10041.jpg',
 'cat.10042.jpg',
 'cat.10043.jpg',
 'cat.10044.jpg',
 'cat.10045.jpg',
 'cat.10046.jpg',
 'cat.1004

In [0]:
def copy_files(prefix_str, range_start, range_end, target_dir):
  image_paths = [os.path.join(work_dir, 'train', prefix_str + '.' + str(i) + '.jpg')
                 for i in range(range_start, range_end)]
  dest_dir = os.path.join(work_dir, 'data', target_dir, prefix_str)
  os.makedirs(dest_dir)
  for image_path in image_paths:
    shutil.copy(image_path, dest_dir)

In this example, we will use only 1000 images of cats and dogs for training and 400 images of cats and dogs for testing, respectively.

In [0]:
copy_files('dog', 0, 1000, 'train')
copy_files('cat', 0, 1000, 'train')

In [0]:
copy_files('dog', 1000, 1400, 'test')
copy_files('cat', 1000, 1400, 'test')

##Build a CNN Model

In [0]:
import numpy as np


In [0]:
work_dir = './data/ch2/data/'
image_height, image_width = 150, 150
train_dir = os.path.join(work_dir, 'train')
test_dir = os.path.join(work_dir, 'test')
no_classes = 2
no_validation = 800
epochs = 2
batch_size = 200
no_train = 2000
no_test = 800
input_shape = (image_height, image_width, 3)
epoch_steps = no_train // batch_size
test_steps = no_test // batch_size

In [0]:
# reset graph
tf.reset_default_graph()
tf.keras.backend.clear_session()

In [0]:
def simple_cnn(input_shape):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Conv2D(filters=64,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   input_shape=input_shape)
           )
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Conv2D(filters=128,
                                   kernel_size=(3,3),
                                   activation='relu')
           )
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
  model.add(tf.keras.layers.Dropout(rate=0.4))
  model.add(tf.keras.layers.Dense(units=no_classes, activation='softmax'))
  
  model.summary()
  model.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
  return model

In [17]:
simple_cnn_model = simple_cnn(input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 165888)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              169870336 
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
__________

In [0]:
generator_train = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1./255)
generator_test = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1./255)

In [22]:
train_images = generator_train.flow_from_directory(train_dir,
                                                   batch_size=batch_size,
                                                   target_size=(image_width, image_height))
test_images = generator_test.flow_from_directory(test_dir,
                                                 batch_size=batch_size,
                                                 target_size=(image_width, image_height))

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [23]:
simple_cnn_model.fit_generator(train_images,
                               steps_per_epoch=epoch_steps,
                               epochs=epochs,
                               validation_data=test_images,
                               validation_steps=test_steps)

Epoch 1/2
10/10 [==============================] - 26s 3s/step - loss: 6.8504 - acc: 0.4890 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/2
10/10 [==============================] - 15s 1s/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000


This model fits the data from the generator of training images. The
number of epochs is defined from training, and validation data is
passed for getting the performance of the model overtraining. This
`fit_generator` enables parallel processing of data and model training.
The CPU performs the rescaling while the GPU can perform the
model training. This gives the high efficiency of computing resources.

#Augment the Dataset
Data augmentation gives ways to increase the size of the dataset. Data
augmentation introduces noise during training, producing robustness in the
model to various inputs. This technique is useful in scenarios when the dataset is
small and can be combined and used with other techniques.

In [0]:
generator_train = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  horizontal_flip=True,
                                                                  zoom_range=0.3,
                                                                  shear_range=0.3)

This will change the zoom range, and shear effect of the training images and also flip the training images horizontally.

In [0]:
generator_test = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)

In [27]:
train_images = generator_train.flow_from_directory(train_dir,
                                                   batch_size=batch_size,
                                                   target_size=(image_width,image_height))
test_images = generator_test.flow_from_directory(test_dir,
                                                 batch_size=batch_size,
                                                 target_size=(image_width,image_height))

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [28]:
simple_cnn_model.fit_generator(train_images,
                               steps_per_epoch=epoch_steps,
                               epochs=epochs,
                               validation_data=test_images,
                               validation_steps=test_steps)

Epoch 1/2
10/10 [==============================] - 28s 3s/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/2
10/10 [==============================] - 25s 2s/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000


Notice how the accuracy increases as we apply data augmentation here.

#Transfer Learning or Fine-Tuning of a Model
Transfer learning is the process of learning from a pre-trained model that was
trained on a larger dataset. Training a model with random initialization often
takes time and energy to get the result. Initializing the model with a pre-trained
model gives faster convergence, saving time and energy. These models that are
pre-trained are often trained with carefully chosen hyperparameters.

Either the several layers of the pre-trained model can be used without any
modification, or can be bit trained to adapt to the changes.

In [0]:
# reset graph
tf.reset_default_graph()
tf.keras.backend.clear_session()

In [0]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [0]:
model = tf.keras.applications.VGG16(include_top=False)
model.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [52]:
train_images = generator.flow_from_directory(train_dir,
                                             batch_size=batch_size,
                                             target_size=(image_width,image_height),
                                             class_mode=None,
                                             shuffle=False)
train_bottleneck_features = model.predict_generator(train_images, epoch_steps)

Found 2000 images belonging to 2 classes.


In [53]:
test_images = generator.flow_from_directory(test_dir,
                                            batch_size=batch_size,
                                            target_size=(image_width,image_height),
                                            class_mode=None,
                                            shuffle=False)
test_bottleneck_features = model.predict_generator(test_images, test_steps)

Found 800 images belonging to 2 classes.


In [0]:
train_labels = np.array([0] * int(no_train / 2) + [1] * int(no_train / 2))
test_labels = np.array([0] * int(no_test / 2) + [1] * int(no_test / 2))

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=train_bottleneck_features.shape[1:]))
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1, activation='softmax'))
model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [56]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              8389632   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1025      
Total params: 8,390,657
Trainable params: 8,390,657
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.fit(train_bottleneck_features,
          train_labels,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test_bottleneck_features, test_labels))

Train on 2000 samples, validate on 800 samples
Epoch 1/2
2000/2000 [==============================] - 1s 483us/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/2
2000/2000 [==============================] - 0s 108us/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000


This gives a different approach to training the model and is useful
when the training data is low. This is often a faster method to train a
model. Only the final activations of the pre-trained model are used to
adapt to the new task.

#Fine-Tune Several Layers in Deep Learning
A pre-trained model can be loaded and only a few layers can be trained. This
approach works better when the given problem is very different from the images
that the model is trained upon. **Fine-tuning** is a common practice in deep
learning. This gives advantages when the dataset is smaller. The optimization
also can be obtained faster.

Training a deep network on a small dataset results in overfitting. This kind of
overfitting can also be avoided using the fine-tuning procedure. The model
trained on a bigger dataset should be also similar, as we are hoping that the
activations and features are similar to the smaller dataset.

#Choose the right model
There are a lot of options for architectures. Based on the flexibility of
deployment, you can choose the model. Remember that convolution is smaller
and slower, but dense layers are bigger and faster. There is a trade-off between
size, runtime, and accuracy. It is advisable to test out all the architectures before
the final decision. Some models may work better than others, based on the
application. You can reduce the input size to make the inference faster.

##Underfitting and Overfitting
The model may be sometimes too big or too small for the problem. This could be
classified as underfitting or overfitting, respectively. Underfitting happens when
the model is too small and can be measured when training accuracy is less.
Overfitting happens when the model is too big and there is a large gap between
training and testing accuracies.

Underfitting can be solved by the following methods:
* Getting more data
* Trying out a bigger model
* If the data is small, try transfer learning techniques or do data augmentation

Overfitting can be solved by the following methods:
* Regularizing using techniques such as dropout and batch normalization
* Augmenting the dataset

##Gender and Age Detection from Face
Applications may require gender and age detection from a face. The face image
can be obtained by face detectors. The cropped images of faces can be supplied
as training data, and the similar cropped face should be given for inference.
Based on the required inference time, OpenCV, or CNN face detectors can be
selected. For training, Inception or ResNet can be used. If the required inference
time is much less because it is a video, it's better to use three convolutions
followed by two fully connected layers. Note that there is usually a huge class
imbalance in age datasets, hence using a different metric like precision and recall
will be helpful.

##Apparel Models
Fine-tuning of apparel models is a good choice. Having multiple softmax layers
that classify attributes will be useful here. The attributes could be a pattern,
color, and so

##Brand Safety
Training bottleneck layers with Support Vector Machine (SVM) is a good
option as the images can be quite different among classes. This is typically used
for content moderation to help avoid images that are explicit.